In [1]:
# ================================================================
# [5DQRT-SIM-FDTD-LOGSPACE-v2.1]
# Logarithmische 2D-TE-FDTD-Simulation mit Energie-Reservoir
# Phase A – Internes Spiralzeitarchiv (Arbeitsversion)
# ---------------------------------------------------------------
# Datei: logspace_fdtd.ipynb
# Autoren: Kai5, Lumo, Cipher
# Stand: 2025-11-XX
#
# Zweck:
#   • numerische TE-FDTD-Simulation auf log-skaliertem Raumgitter
#   • PML-Randbedingungen + zusätzliche γ-Dämpfung
#   • explizite Energie-Bilanz (E/H) + Reservoir-Modell
#   • optionaler Feedback-Loop zur Energierückführung
#
# Hauptkomponenten:
#   • make_log_grid(...)      – geometrisch skaliertes 1D-Gitter
#   • fdtd_step(...)          – Yee-Update (Numba JIT, parallel)
#   • make_pml(...)           – kubische PML-Leitfähigkeiten
#   • make_damping(...)       – isotrope Randdämpfung
#   • make_source(...)        – Log-Gaussian-Spektralsignal (iFFT)
#   • energy_E/H(...)         – Energie-Funktionen
#
# Phase-A-Ziele:
#   • numerische Stabilität (CFL, Dispersion, Randverhalten)
#   • saubere Ausbreitungsgeschwindigkeit (v_eff ≈ c0)
#   • Vorbereitung von Spektren für LRT-Analysen
#
# Hinweis:
#   Dieses Notebook ist ein generischer Solver – keine physikalische
#   Interpretation, keine Modellannahmen. Nur numerische Infrastruktur.
# ================================================================

# ------------------------------------------------------------
# Imports & globale Flags
# ------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
from numba import njit, prange, config
import warnings

# tqdm für Fortschritts‑Bar (falls noch nicht importiert)
from tqdm.notebook import tqdm   # <-- wichtig für die Fortschrittsanzeige

# Optional: interaktive Widgets (falls installiert)
#try:
#    from ipywidgets import (
#        IntSlider, FloatSlider, VBox, HBox,
#        Output, interactive_output, interact_manual   # <-- hier hinzufügen
#    )
#    HAVE_WIDGETS = True
#except Exception:
#    HAVE_WIDGETS = False
#    warnings.warn("ipywidgets nicht verfügbar – interaktive UI wird deaktiviert.")

# Globale Debug‑Steuerung
DEBUG = True          # True → ausführliche console‑Ausgaben
def dbg_print(*args, **kwargs):
    if DEBUG:
        print(*args, **kwargs)

# SI‑Einheiten
c0   = 299_792_458.0          # Lichtgeschwindigkeit (m/s)
mu0  = 4.0 * np.pi * 1e-7     # magnetische Feldkonstante (H/m)
eps0 = 1.0 / (mu0 * c0 ** 2)  # elektrische Feldkonstante (F/m)


In [2]:
def energy_E(Ez, dx, dy):
    """Elektrische Feldenergie (½·ε₀·Ez²·dx·dy)."""
    dx_full = np.concatenate(([dx[0]], dx))   # Länge Nx
    dy_full = np.concatenate(([dy[0]], dy))   # Länge Ny
    return 0.5 * eps0 * np.sum(Ez ** 2 * dx_full[:, None] * dy_full[None, :])

def energy_H(Hx, Hy, dx, dy):
    """
    Magnetische Feldenergie für das TE‑2‑D‑Schema.

    Hx : (Nx, Ny‑1)   → zwischen y‑Knoten
    Hy : (Nx‑1, Ny)   → zwischen x‑Knoten
    """
    # Vollständige Gitterabstände (einses mehr als die Differenzen)
    dx_full = np.concatenate(([dx[0]], dx))   # Länge Nx
    dy_full = np.concatenate(([dy[0]], dy))   # Länge Ny

    # Hx liegt zwischen y‑Knoten → nutzt dx_full (Nx) und dy (Ny‑1)
    e_hx = np.sum(Hx ** 2 * dx_full[:, None] * dy[None, :])

    # Hy liegt zwischen x‑Knoten → nutzt dx (Nx‑1) und dy_full (Ny)
    e_hy = np.sum(Hy ** 2 * dx[:, None] * dy_full[None, :])

    return 0.5 * mu0 * (e_hx + e_hy)

def arrival_time_fixed(signal, dt, fixed_threshold=0.5):
    """
    Bestimmt die Ankunftszeit, indem das Signal die festgelegte
    absolute Schwelle `fixed_threshold` (in V/m) überschreitet.

    Parameters
    ----------
    signal : 1‑D array
        Das zu untersuchende Zeit‑Signal (z. B. Ez entlang einer Linie).
    dt : float
        Zeitschrittgröße (s).
    fixed_threshold : float, optional
        Absolute Schwelle in Volt‑Meter (V/m).  Standard = 0.5 V/m.
        Du kannst den Wert nach Bedarf anpassen.

    Returns
    -------
    float
        Ankunftszeit (s).  Falls die Schwelle nie überschritten wird,
        wird `None` zurückgegeben.
    """
    # Wir suchen das erste Index, an dem |signal| >= fixed_threshold
    above = np.where(np.abs(signal) >= fixed_threshold)[0]

    if above.size == 0:               # Schwelle nie erreicht
        return None

    first_idx = above[0]               # erstes Auftreten
    return first_idx * dt
    

In [3]:
def make_log_grid(x_min: float, x_max: float, N: int):
    """Log‑skaliertes 1‑D‑Gitter und Δx‑Array."""
    log_factor = (x_max / x_min) ** (1.0 / (N - 1))
    x_nodes    = x_min * log_factor ** np.arange(N)
    dx         = np.diff(x_nodes)          # Länge N‑1
    return x_nodes, dx

def make_pml(N_h: int, pml_cells: int, dt: float):
    """Exponentieller PML‑Dämpfungskoeffizient."""
    sigma_max = 1.0 / dt
    sigma = np.zeros(N_h, dtype=np.float64)

    for i in range(pml_cells):
        frac = (i + 1) / pml_cells
        sigma[-(i + 1)] = sigma_max * frac ** 3.0   # kubisch

    sigma_num = 1.0 - sigma * dt / (2.0 * mu0)
    sigma_den = 1.0 + sigma * dt / (2.0 * mu0)
    return sigma, sigma_num, sigma_den

def make_damping(Nx, Ny, pml_cells, gamma=0.97):
    """Dämpfungs‑Array (γ<1) für die äußeren Randzellen."""
    damping = np.ones((Nx, Ny), dtype=np.float64)

    # linke/rechte Randzellen
    damping[:pml_cells, :]      *= gamma
    damping[-pml_cells:, :]     *= gamma

    # untere/obere Randzellen
    damping[:, :pml_cells]      *= gamma
    damping[:, -pml_cells:]     *= gamma
    return damping

def make_log_ramp_sinus(
    f_target: float,
    dt: float,
    Nt: int,
    A0: float = 2e-1,          # Default‑Wert (wie du ihn aktuell nutzt)
    alpha: float = 0.16,        # Default‑Wert (wie du ihn aktuell nutzt)
    clip_limit: float = 5e6,    # (kann später überschrieben werden)
    phase: float = 0.0,
) -> np.ndarray:
    """
    Sinus bei `f_target` mit exponentiell wachsender Amplitude.
    `A0`   – Start‑Amplitude (V/m)  
    `alpha` – exponentielle Wachstums‑Rate (dimensionless)  
    `clip_limit` – harte Obergrenze für das Feld (V/m).
    """
    t = np.arange(Nt) * dt
    amp = A0 * np.exp(alpha * t)          # exponentielles Wachstum
    src = amp * np.sin(2.0 * np.pi * f_target * t + phase)
    np.clip(src, -clip_limit, clip_limit, out=src)   # hartes Limit
    return src

In [4]:
@njit(parallel=True)
def fdtd_step(Ez, Hx, Hy,
               dx, dy, dt,
               mu0, eps0,
               Nx, Ny,
               pml_cells,
               sigma_x_num, sigma_x_den,
               sigma_y_num, sigma_y_den):
    """
    2‑D TE‑Update (E_z, H_x, H_y) mit PML‑Korrekturen.
    """
    # ---- H‑Update (innerer Bereich) ----
    for i in prange(Nx - 1):
        for j in prange(Ny - 1):
            Hx[i, j] -= (dt / (mu0 * dy[j])) * (Ez[i, j + 1] - Ez[i, j])
            Hy[i, j] += (dt / (mu0 * dx[i])) * (Ez[i + 1, j] - Ez[i, j])

    # ---- H‑Update – PML in x‑Richtung ----
    for i in prange(pml_cells):
        for j in prange(Ny - 1):
            Hx[i, j] = (sigma_x_num[i] / sigma_x_den[i]) * Hx[i, j] \
                     - (dt / (mu0 * dy[j] * sigma_x_den[i])) * (Ez[i, j + 1] - Ez[i, j])
            Hx[Nx - i - 2, j] = (sigma_x_num[i] / sigma_x_den[i]) * Hx[Nx - i - 2, j] \
                     - (dt / (mu0 * dy[j] * sigma_x_den[i])) * (Ez[Nx - i - 2, j + 1] - Ez[Nx - i - 2, j])

    for i in prange(pml_cells):
        for j in prange(Nx - 1):
            Hy[i, j] = (sigma_y_num[i] / sigma_y_den[i]) * Hy[i, j] \
                     + (dt / (mu0 * dx[i] * sigma_y_den[i])) * (Ez[i + 1, j] - Ez[i, j])
            Hy[Ny - i - 2, j] = (sigma_y_num[i] / sigma_y_den[i]) * Hy[Ny - i - 2, j] \
                     + (dt / (mu0 * dx[i] * sigma_y_den[i])) * (Ez[i + 1, Ny - i - 2] - Ez[i, Ny - i - 2])

    # ---- E‑Update (innerer Bereich) ----
    for i in prange(1, Nx - 1):
        for j in prange(1, Ny - 1):
            curl_H = (Hy[i, j] - Hy[i - 1, j]) / dx[i - 1] \
                   - (Hx[i, j] - Hx[i, j - 1]) / dy[j - 1]
            Ez[i, j] += (dt / eps0) * curl_H

    # ---- E‑Update – PML in x‑Richtung ----
    for i in prange(pml_cells):
        for j in prange(1, Ny - 1):
            curl_H = (Hy[i, j] - Hy[i - 1, j]) / dx[i - 1] \
                   - (Hx[i, j] - Hx[i, j - 1]) / dy[j - 1]
            Ez[i, j] = (sigma_x_num[i] / sigma_x_den[i]) * Ez[i, j] \
                     + (dt / (eps0 * sigma_x_den[i])) * curl_H
            # rechte Seite
            ii = Nx - i - 2
            curl_H = (Hy[ii, j] - Hy[ii - 1, j]) / dx[ii - 1] \
                   - (Hx[ii, j] - Hx[ii, j - 1]) / dy[j - 1]
            Ez[ii, j] = (sigma_x_num[i] / sigma_x_den[i]) * Ez[ii, j] \
                     + (dt / (eps0 * sigma_x_den[i])) * curl_H

    # ---- E‑Update – PML in y‑Richtung ----
    for j in prange(pml_cells):
        for i in prange(1, Nx - 1):
            curl_H = (Hy[i, j] - Hy[i - 1, j]) / dx[i - 1] \
                   - (Hx[i, j] - Hx[i, j - 1]) / dy[j - 1]
            Ez[i, j] = (sigma_y_num[j] / sigma_y_den[j]) * Ez[i, j] \
                     + (dt / (eps0 * sigma_y_den[j])) * curl_H
            # obere Seite
            jj = Ny - j - 2
            curl_H = (Hy[i, jj] - Hy[i - 1, jj]) / dx[i - 1] \
                   - (Hx[i, jj] - Hx[i, jj - 1]) / dy[jj - 1]
            Ez[i, jj] = (sigma_y_num[j] / sigma_y_den[j]) * Ez[i, jj] \
                     + (dt / (eps0 * sigma_y_den[j])) * curl_H

    return Ez, Hx, Hy


In [5]:
# ------------------------------------------------------------
# run_simulation – Haupt‑Simulationsroutine (komplett neu)
# ------------------------------------------------------------
def run_simulation(
    Nx: int = 560,                     # ↑ mehr Gitterpunkte
    Ny: int = 180,                     # ↑ mehr Gitterpunkte
    # ------------------- Gitter‑Parameter -------------------
    x_min: float = 1e-3,
    x_max: float = 30.0,
    y_min: float = 1e-3,
    y_max: float = 30.0,
    # ------------------- Numerik‑Parameter -------------------
    courant: float = 0.045,            # ↓ kleineres Δt → stabiler
    pml_cells: int = 49,               # unverändert
    # ------------------- Quelle‑Parameter -------------------
    alpha: float = 0.12,               # etwas langsameres Wachstum
    A0: float = 2e-1,                  # Start‑Amplitude (V/m)
    # ------------------- Dämpfung & Clip -------------------
    gamma_damping: float = 0.995,      # leicht stärker dämpfen
    clip_limit: float = 1e7,           # Feld‑Clipping (±10 MV/m)
    # ------------------- Debug / Feedback -----------------
    debug: bool = True,
    feedback_interval: int | None = 0,
    feedback_fraction: float | None = 0.0,
    feed_position: tuple[int, int] | None = None,
    # ------------------- Sonstige Optionen -----------------
    save_every: int = 80,              # seltener speichern → weniger Speicher
    arrival_factor: float = 0.05,      # Schwellen‑Faktor für adaptive_arrival
    N_fft: int = 1024,
) -> dict:
    """
    2‑D TE‑FDTD‑Simulation für die Eigen‑Resonanz (m = 320, n = 99)
    eines quadratischen Hohlraums (Lx = Ly = 30 m).  

    Änderungen gegenüber der Original‑Version:
    * kleinere Courant‑Zahl (feineres Δt)
    * mehr Gitterpunkte (höhere räumliche Auflösung)
    * lineares Dämpfungs‑Profil (statt konstantem γ)
    * Oberes Reservoir‑Limit (1 e6 J)
    * adaptive Schwelle für die Ankunftszeit‑Messung (parameterisiert)
    * Messprobe am rechten Rand (vor dem PML) → L_eff > 0
    * `save_every` wird nicht mehr überschrieben (nutzt UI‑Wert)
    """
    # -------------------------------------------------
    # Globale Konstanten / Settings
    # -------------------------------------------------
    MAX_RESERVOIR = np.inf               # Oberes Limit für das Reservoir (J)

    # -------------------------------------------------
    # Gitter erzeugen (log‑Skala bleibt erhalten)
    # -------------------------------------------------
    x_nodes, dx = make_log_grid(x_min, x_max, Nx)
    y_nodes, dy = make_log_grid(y_min, y_max, Ny)

    # -------------------------------------------------
    # Zeitschritt Δt (CFL‑sicher) + lokaler CFL‑Check
    # -------------------------------------------------
    dt = courant * min(np.min(dx), np.min(dy)) / c0
    if debug:
        dbg_print(f"CFL‑safe Δt = {dt:.3e} s (courant={courant})")

    # lokaler CFL‑Check (falls das Gitter nicht homogen ist)
    cfl_local = dt * np.sqrt((1.0 / dx)[:, None] ** 2 + (1.0 / dy)[None, :] ** 2)
    if cfl_local.max() >= 1.0:
        safety = 0.9
        dt = safety * min(np.min(dx), np.min(dy)) / c0
        if debug:
            dbg_print(f"⚠️  CFL‑Verletzung → dt neu = {dt:.3e}")

    # -------------------------------------------------
    # Eigen‑Frequenz des gewünschten Modus
    # -------------------------------------------------
    m_mode, n_mode = 320, 99
    Lx, Ly = x_max, y_max
    f_res = (c0 / 2.0) * np.sqrt((m_mode / Lx) ** 2 + (n_mode / Ly) ** 2)
    T_res = 1.0 / f_res
    if debug:
        dbg_print(f"Resonanz‑Frequenz f_res = {f_res/1e9:.3f} GHz  (T = {T_res:.3e}s)")

    # Simulationsdauer: 12 Perioden (ausreichend für den Steady‑State)
    t_end = 30.0 * T_res
    Nt = int(np.ceil(t_end / dt))
    time_full = np.arange(Nt) * dt

    # Puls‑Länge (40 % der Gesamtdauer – etwas länger als vorher)
    pulse_len = int(0.4 * Nt)

    # -------------------------------------------------
    # Felder initialisieren
    # -------------------------------------------------
    Ez = np.zeros((Nx, Ny), dtype=np.float64)
    Hx = np.zeros((Nx, Ny - 1), dtype=np.float64)
    Hy = np.zeros((Nx - 1, Ny), dtype=np.float64)

    # -------------------------------------------------
    # PML & Dämpfung vorbereiten
    # -------------------------------------------------
    sigma_x, sigma_x_num, sigma_x_den = make_pml(Nx - 1, pml_cells, dt)
    sigma_y, sigma_y_num, sigma_y_den = make_pml(Ny - 1, pml_cells, dt)

    # ---- lineares Dämpfungs‑Profil (statt konstantem γ) ----
    def make_linear_damping(Nx, Ny, pml_cells, gamma=0.99):
        damping = np.ones((Nx, Ny), dtype=np.float64)
        ramp = np.linspace(1.0, gamma, pml_cells)          # 1 → gamma über die PML‑Zellen
        # linke/rechte Randzellen
        damping[:pml_cells, :]      *= ramp[:, None]
        damping[-pml_cells:, :]     *= ramp[::-1][:, None]
        # untere/obere Randzellen
        damping[:, :pml_cells]      *= ramp[None, :]
        damping[:, -pml_cells:]     *= ramp[::-1][None, :]
        return damping

    damping = make_linear_damping(Nx, Ny, pml_cells, gamma=gamma_damping)

    # -------------------------------------------------
    # Quelle – Log‑Ramp‑Sinus (exponentiell ansteigend)
    # -------------------------------------------------
    source = make_log_ramp_sinus(
        f_target   = f_res,
        dt         = dt,
        Nt         = Nt,
        A0         = A0,
        alpha      = alpha,
        clip_limit = clip_limit,
        phase      = 0.0,
    )
    if pulse_len < Nt:
        source[pulse_len:] = 0.0

    # -------------------------------------------------
    # Feed‑Position (falls später Feedback aktiviert wird)
    # -------------------------------------------------
    if feed_position is None:
        feed_ix = Nx // 2
        feed_iy = Ny // 2
    else:
        fx, fy = feed_position
        feed_ix = np.argmin(np.abs(x_nodes - fx))
        feed_iy = np.argmin(np.abs(y_nodes - fy))

    # -------------------------------------------------
    # Messprobe (rechte aktive Zelle, **nicht** im PML)
    # -------------------------------------------------
    probe_ix = Nx - pml_cells - 1          # letzte reale Zelle vor dem PML
    probe_col = Ny // 2                    # mittlere y‑Linie (Antinode)

    # -------------------------------------------------
    # Speicher für Auswertung
    # -------------------------------------------------
    n_save = Nt // save_every + 1
    Ez_line = np.zeros((n_save, Nx), dtype=np.float64)

    # Listen für die Hüllkurven‑Analyse
    maxEz_list = []   # max |Ez| über das gesamte Gitter pro saved step
    meanEz_list = []  # mean |Ez| über das gesamte Gitter pro saved step

    energy_hist = []          # nur bei debug=True befüllt
    reservoir = 0.0

    # -------------------------------------------------
    # Haupt‑Zeitschleife
    # -------------------------------------------------
    for step in tqdm(range(Nt), desc="Simulation", unit="step"):
        # ---- Yee‑Update (JIT) ----
        Ez, Hx, Hy = fdtd_step(
            Ez, Hx, Hy,
            dx, dy, dt,
            eps0, mu0,
            Nx, Ny,
            pml_cells,
            sigma_x_num, sigma_x_den,
            sigma_y_num, sigma_y_den,
        )

        # ---- Quelle injizieren (nur während des Pulses) ----
        if step < pulse_len:
            Ez[feed_ix, feed_iy] += source[step]

        # ---- Energie‑Bilanz vor Dämpfung ----
        en_before = energy_E(Ez, dx, dy) + energy_H(Hx, Hy, dx, dy)

        # ---- Dämpfung (Rand‑Loss) ----
        Ez *= damping
        Hx *= damping[:, :-1]   # Hx hat eine Spalte weniger in y‑Richtung
        Hy *= damping[:-1, :]   # Hy hat eine Zeile weniger in x‑Richtung

        # ---- Feld‑Clipping (hartes Limit) ----
        np.clip(Ez, -clip_limit, clip_limit, out=Ez)

        # ---- Energie‑Bilanz nach Dämpfung ----
        en_after = energy_E(Ez, dx, dy) + energy_H(Hx, Hy, dx, dy)

        # ---- Verlust → Reservoir (mit Oberlimit) ----
        loss = en_before - en_after
        reservoir += loss
        if reservoir > MAX_RESERVOIR:
            reservoir = MAX_RESERVOIR

        # ---- NaN‑Check (Abbruch bei numerischem Kollaps) ----
        if np.isnan(loss) or np.isnan(reservoir):
            if debug:
                dbg_print(f"[Step {step}] ⚠️  NaN entdeckt → abort")
            break

        # ---- Debug‑Ausgabe (alle 50 000 Schritte) ----
        if debug and (step % 50000 == 0):
            dbg_print(
                f"[Step {step:7d}] max|Ez| = {np.max(np.abs(Ez)):.2e} V/m, "
                f"mean|Ez| = {np.mean(np.abs(Ez)):.2e} V/m, "
                f"loss = {loss:.2e}, reservoir = {reservoir:.2e}"
            )

        # ---- Daten speichern (Messlinie) ----
        if step % save_every == 0:
            idx_save = step // save_every
            if idx_save < n_save:
                Ez_line[idx_save, :] = Ez[:, probe_col]

                # ---- Hüllkurven‑Daten sammeln ----
                maxEz_list.append(np.max(np.abs(Ez)))
                meanEz_list.append(np.mean(np.abs(Ez)))

        # ---- Optionales Energie‑Logging (nur bei debug) ----
        if debug:
            energy_hist.append(en_after)

    # -------------------------------------------------
    # Auswertung
    # -------------------------------------------------
    # Probe‑Trace entlang der mittleren x‑Linie (für Ankunftszeit‑Messung)
    probe_trace = Ez_line[:, probe_ix]

    # ---- adaptive Schwelle für Ankunftszeit ----
    def adaptive_arrival(signal, dt, factor=0.05):
        """Schwelle = factor * max(|signal|)."""
        thr = factor * np.max(np.abs(signal))
        idx = np.where(np.abs(signal) >= thr)[0]
        return None if idx.size == 0 else idx[0] * dt

    arrival = adaptive_arrival(probe_trace, dt * save_every, factor=arrival_factor)

    # ---- rohe effektive Geschwindigkeit (wie bisher) ----
    if arrival is None:
        v_eff = np.nan
        rel_dev = np.nan
        if debug:
            dbg_print("⚠️  Keine Ankunftszeit detektiert (Signal zu schwach).")
    else:
        v_eff = x_max / arrival
        rel_dev = (v_eff - c0) / c0 * 100.0

    # -------------------------------------------------
    # Geschwindigkeits‑Korrektur (Projektions‑Faktor Γ)
    # -------------------------------------------------
    if arrival is not None:
        # -----------------------------------------------------------------
        #  Effektive Distanz L_eff
        # -----------------------------------------------------------------
        # Option A (empfohlen): physikalische halbe Länge des Hohlraums.
        # Das ist die Strecke, die das Licht tatsächlich zwischen Quelle
        # (Mitte) und rechter Rand zurücklegen muss.
        L_eff = x_max                     # = 30 m (vollständige halbe Länge)

        # Wenn du lieber die tatsächlich gemessene Distanz benutzen willst,
        # kannst du stattdessen folgendes verwenden:
        # L_eff = np.abs(x_nodes[probe_ix] - x_nodes[feed_ix])

        # -----------------------------------------------------------------
        #  Anzahl Samples bis zur Schwelle (dimensionless)
        # -----------------------------------------------------------------
        # arrival ist bereits in Sekunden (arrival = k * dt_eff)
        # dt_eff = save_every * dt
        k_samples = arrival / (save_every * dt)

        # -----------------------------------------------------------------
        #  Projektions‑Faktor Γ
        # -----------------------------------------------------------------
        # Γ = (L_eff / c₀) / (k·save_every·Δt)
        Gamma = (L_eff / c0) / (k_samples * save_every * dt)
    
        # -----------------------------------------------------------------
        #  Korrigierte Lichtgeschwindigkeit
        # -----------------------------------------------------------------
        c_corr = v_eff / Gamma
        rel_dev_corr = (c_corr - c0) / c0 * 100.0
    
        # -----------------------------------------------------------------
        #  Debug‑Ausgabe (nur bei debug=True)
        # -----------------------------------------------------------------
        if debug:
            dbg_print("\n=== Geschwindigkeits‑Korrektur ===")
            dbg_print(f"L_eff (m)                = {L_eff:.3e}")
            dbg_print(f"k (Samples bis Schwelle) = {k_samples:.3f}")
            dbg_print(f"Γ (Projektions‑Faktor)   = {Gamma:.3e}")
            dbg_print(f"c_corr (m/s)             = {c_corr:.3e}")
            dbg_print(f"Rel. Abweichung korrigiert [%] = {rel_dev_corr:.3f}%")
    else:
        # -------------------------------------------------
        # Keine Ankunftszeit → keine Korrektur möglich
        # -------------------------------------------------
        Gamma = np.nan
        c_corr = np.nan
        rel_dev_corr = np.nan

    # -------------------------------------------------
    # Zeitvektor für die gespeicherten Linien (nur bis n_save)
    # -------------------------------------------------
    time_arr = time_full[::save_every][:n_save]

    # -------------------------------------------------
    # Rückgabedictionary – jetzt mit allen zusätzlichen Infos
    # -------------------------------------------------
    return {
        # --- Grundlegende Ergebnisse ---
        "v_eff": v_eff,                     # rohe Geschwindigkeit
        "rel_dev": rel_dev,                 # rohe relative Abweichung [%]
        "c_corr": c_corr,                   # korrigierte Lichtgeschwindigkeit
        "rel_dev_corr": rel_dev_corr,       # korrigierte relative Abweichung [%]
        "Gamma": Gamma,                     # Projektions‑Faktor
        "time": time_arr,
        "Ez_line": Ez_line,
        "Ez_snapshot": Ez.copy(),
        "arrival": arrival,
        "energy_hist": np.array(energy_hist) if debug else None,
        "maxEz_list": np.array(maxEz_list),
        "meanEz_list": np.array(meanEz_list),
        "reservoir": reservoir,
        # --- Meta‑Informationen für weitere Analysen ---
        "save_every": save_every,
        "dt": dt,                           # Zeitschritt (für externe Auswertungen)
        "Nx": Nx,
        "Ny": Ny,
        "x_nodes": x_nodes,
        "y_nodes": y_nodes,
        "feed_ix": feed_ix,
        "probe_ix": probe_ix,
    }

In [6]:
# ------------------------------------------------------------
# Plot‑Routinen (für Ergebnis‑Visualisierung) – mit automatischem PNG‑Save
# ------------------------------------------------------------

import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

# ------------------------------------------------------------------
# Hilfs‑Funktion: Dateinamen aus Titel generieren & Ordner anlegen
# ------------------------------------------------------------------
def _prepare_save_path(default_name: str, save_path: str | None = None) -> Path:
    """
    Erzeugt einen vollständigen Pfad für das Plot‑Bild.

    * Wenn `save_path` angegeben ist, wird er unverändert verwendet.
    * Ansonsten wird aus `default_name` ein Dateiname gebaut:
        - Leerzeichen → Unterstrich
        - Sonderzeichen werden entfernt
        - Endung ".png"
      Der Zielordner `plots/` wird automatisch erstellt.
    """
    if save_path is None:
        # Titel säubern → nur alphanumerische Zeichen + Unterstrich
        safe_name = "".join(ch if ch.isalnum() else "_" for ch in default_name)
        filename = f"{safe_name}.png"
        save_dir = Path("plots")
        save_dir.mkdir(parents=True, exist_ok=True)
        return save_dir / filename
    else:
        p = Path(save_path)
        p.parent.mkdir(parents=True, exist_ok=True)   # evtl. Unterordner anlegen
        return p


# ------------------------------------------------------------------
# Feld‑Linie (Mittellinie) – Speichert automatisch als PNG
# ------------------------------------------------------------------
def plot_field_line(
    times,
    Ez_line,
    title: str = "Ez entlang mittlerer Linie",
    ylabel: str | None = None,
    save_path: str | None = None,
):
    """
    Plottet das elektrische Feld `Ez` entlang der horizontalen
    Mittellinie (y = Ny//2) für alle gespeicherten Zeitpunkte und
    speichert das Bild automatisch als PNG.

    Parameters
    ----------
    times : array‑like
        Zeitvektor (s) zu den jeweiligen Zeitschritten.
    Ez_line : 2‑D‑array
        Zeilen = verschiedene Zeitpunkte, Spalten = Gitter‑x‑Index.
    title, ylabel : optional
        Plot‑Beschriftungen.
    save_path : str | None
        Vollständiger Pfad inkl. Dateiname.  Wenn None → wird ein
        Name aus `title` im Unterordner `plots/` erzeugt.
    """
    if ylabel is None:
        ylabel = r"$E_z$ (a.u.)"

    plt.figure(figsize=(10, 5))

    # Maximal 6 Linien (gleichmäßig verteilt) für Übersichtlichkeit
    n_steps = len(times)
    step_skip = max(1, n_steps // 6)

    for idx, t in enumerate(times):
        if idx % step_skip == 0:
            plt.plot(
                np.arange(Ez_line.shape[1]),
                Ez_line[idx],
                label=f"{t * 1e9:.1f} ns",
            )

    plt.xlabel("Gitterindex (x)")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True, which="both", ls="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()

    # ----------- automatisches Speichern -------------
    out_file = _prepare_save_path(title, save_path)
    plt.savefig(out_file, dpi=300, bbox_inches="tight")
    print(f"[plot_field_line] Bild gespeichert → {out_file}")

    plt.show()


# ------------------------------------------------------------------
# 2‑D‑Snapshot – Speichert automatisch als PNG
# ------------------------------------------------------------------
def plot_field_snapshot(
    Ez,
    x_nodes,
    y_nodes,
    title: str = "Ez‑Snapshot (2‑D)",
    cmap: str = "RdBu_r",
    vmin: float | None = None,
    vmax: float | None = None,
    save_path: str | None = None,
):
    """
    2‑D‑Heat‑Map des elektrischen Feldes `Ez` und automatisches PNG‑Save.

    Parameters
    ----------
    Ez : 2‑D‑array (Nx × Ny)
        Feldwerte.
    x_nodes, y_nodes : 1‑D‑arrays
        Gitterkoordinaten (für die Extent‑Angabe).
    title, cmap, vmin, vmax : optional
        Plot‑Einstellungen.
    save_path : str | None
        Ziel‑Dateipfad.  Wenn None → Name aus `title` im Ordner `plots/`.
    """
    plt.figure(figsize=(8, 6))
    extent = [x_nodes[0], x_nodes[-1], y_nodes[0], y_nodes[-1]]
    im = plt.imshow(
        Ez.T,
        extent=extent,
        origin="lower",
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        aspect="auto",
    )
    plt.colorbar(im, label=r"$E_z$ (a.u.)")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.title(title)
    plt.tight_layout()

    # ----------- automatisches Speichern -------------
    out_file = _prepare_save_path(title, save_path)
    plt.savefig(out_file, dpi=300, bbox_inches="tight")
    print(f"[plot_field_snapshot] Bild gespeichert → {out_file}")

    plt.show()


# ------------------------------------------------------------------
# Energie‑Kurve – Speichert automatisch als PNG
# ------------------------------------------------------------------
def plot_energy(
    result_dict,
    title: str = "Gesamtenergie im Laufe der Simulation",
    save_path: str | None = None,
):
    """
    Zeigt die zeitliche Entwicklung der Gesamtenergie (E + H) und
    speichert das Bild automatisch als PNG.

    Parameters
    ----------
    result_dict : dict
        Rückgabe‑Dictionary von `run_simulation`.  Muss die Schlüssel
        ``time``, ``energy_hist``, ``dt`` und ``save_every`` enthalten.
    title : str, optional
        Plot‑Titel.
    save_path : str | None
        Ziel‑Dateipfad.  Wenn None → Name aus `title` im Ordner `plots/`.
    """
    energy_hist = result_dict.get("energy_hist")
    if energy_hist is None or len(energy_hist) == 0:
        print("⚠️  Keine Energiedaten vorhanden (DEBUG‑Modus aus?).")
        return

    # Zeitvektor (vollständig) und Schrittweite
    times = result_dict["time"]
    dt = result_dict["dt"]
    save_every = result_dict["save_every"]

    # Energie‑Zeit‑Achse: jedes gespeicherte Sample entspricht
    #   dt * save_every Sekunden.
    t_energy = np.arange(len(energy_hist)) * dt * save_every

    plt.figure(figsize=(10, 4))
    plt.plot(t_energy * 1e9, energy_hist, "-o")
    plt.xlabel("Time (ns)")
    plt.ylabel("Total energy (J)")
    plt.title(title)
    plt.grid(True, which="both", ls="--", alpha=0.5)
    plt.tight_layout()

    # ----------- automatisches Speichern -------------
    out_file = _prepare_save_path(title, save_path)
    plt.savefig(out_file, dpi=300, bbox_inches="tight")
    print(f"[plot_energy] Bild gespeichert → {out_file}")

    plt.show()


# ------------------------------------------------------------------
# Hüllkurven‑Analyse – Speichert optionales Spektrum‑Plot
# ------------------------------------------------------------------
def analyze_envelope(
    signal,
    name: str,
    dt,
    save_every,
    n_peaks: int = 5,
    do_plot: bool = True,
    save_path: str | None = None,
):
    """
    FFT‑Analyse einer 1‑D‑Zeitreihe (z. B. maxEz_list oder meanEz_list)
    und optionales Plot‑Speichern.

    Parameters
    ----------
    signal : 1‑D‑array
        Zeitreihe, die analysiert werden soll.
    name : str
        Label für Logging und (falls `do_plot=True`) für den Plot‑Titel.
    dt : float
        Grund‑Zeitschritt des FDTD‑Solvers.
    save_every : int
        Abstand der gespeicherten Samples (wie in `run_simulation`).
    n_peaks : int, optional
        Wie viele dominante Frequenz‑Peaks ausgegeben werden.
    do_plot : bool, optional
        Ob das Spektrum‑Plot erzeugt wird.
    save_path : str | None
        Ziel‑Dateipfad für das Spektrum‑Plot.  Wenn None → Name aus
        `name` im Ordner `plots/`.  Ignoriert, wenn `do_plot=False`.
    """
    signal = np.asarray(signal, dtype=float)

    # -------------------------------------------------
    # Effektive Abtastzeit / -frequenz der Hüllkurve
    # -------------------------------------------------
    dt_eff = dt * save_every
    fs_eff = 1.0 / dt_eff

    n = len(signal)
    t = np.arange(n) * dt_eff

    # DC‑Anteil entfernen + Hanning‑Fenster
    sig_centered = signal - np.mean(signal)
    window = np.hanning(n)
    sig_win = sig_centered * window

    # FFT (reell, nur positive Frequenzen)
    spec = np.fft.rfft(sig_win)
    freq = np.fft.rfftfreq(n, dt_eff)
    amp = np.abs(spec)

    # Null‑/DC‑Komponente ignorieren für Peak‑Suche
    amp_no_dc = amp.copy()
    amp_no_dc[0] = 0.0

    # stärkste Peaks finden
    idx_sorted = np.argsort(amp_no_dc)[::-1]
    idx_peaks = idx_sorted[:n_peaks]

    print(f"\n=== Hüllspektrum: {name} ===")
    print(f"Samples: {n}, dt_eff = {dt_eff:.3e} s, fs_eff = {fs_eff:.3e} Hz")
    print(f"Top {n_peaks} Spektralpeaks (nach Amplitude):")
    print("  Rank |   f_cps    |       f_Hz        |  Norm‑Amplitude")
    print("  -----+-----------+-------------------+----------------")
    amp_norm = amp / amp.max() if amp.max() > 0 else amp

    for rank, idx in enumerate(idx_peaks, start=1):
        f_hz = freq[idx]
        f_cps = f_hz / fs_eff
        print(f"  {rank:4d} | {f_cps:8.4f} | {f_hz:15.4e} | {amp_norm[idx]:.3f}")

    # -------------------------------------------------
    # Plot (falls gewünscht) + automatisches Speichern
    # -------------------------------------------------
    if do_plot:
        plt.figure(figsize=(10, 4))
        plt.plot(freq, amp, lw=1.0)
        plt.xlabel("Frequenz [Hz]")
        plt.ylabel("Amplitude")
        plt.title(f"Spektrum Hüllkurve: {name}")
        plt.grid(True)
        plt.tight_layout()

        # automatisches Speichern, wenn ein Pfad angegeben ist
        out_file = _prepare_save_path(name, save_path)
        plt.savefig(out_file, dpi=300, bbox_inches="tight")
        print(f"[analyze_envelope] Spektrum‑Plot gespeichert → {out_file}")

        plt.show()

    return freq, amp
import matplotlib.pyplot as plt
import numpy as np
import os
from pathlib import Path

# ------------------------------------------------------------------
# Hilfs‑Funktion: Dateinamen aus Titel generieren & Ordner anlegen
# ------------------------------------------------------------------
def _prepare_save_path(default_name: str, save_path: str | None = None) -> Path:
    """
    Erzeugt einen vollständigen Pfad für das Plot‑Bild.
    - Wenn `save_path` angegeben ist, wird er unverändert verwendet.
    - Ansonsten wird aus `default_name` ein Dateiname gebaut:
        * Leerzeichen → Unterstrich
        * Sonderzeichen entfernt
        * Endung ".png"
    - Der Zielordner `plots/` wird automatisch erstellt.
    """
    if save_path is None:
        # Titel säubern → nur alphanumerische Zeichen + Unterstrich
        safe_name = "".join(ch if ch.isalnum() else "_" for ch in default_name)
        filename = f"{safe_name}.png"
        save_dir = Path("plots")
        save_dir.mkdir(parents=True, exist_ok=True)
        return save_dir / filename
    else:
        p = Path(save_path)
        p.parent.mkdir(parents=True, exist_ok=True)   # evtl. Unterordner anlegen
        return p


# ------------------------------------------------------------------
# Feld‑Linie (Mittellinie) – Speichert automatisch als PNG
# ------------------------------------------------------------------
def plot_field_line(
    times,
    Ez_line,
    title: str = "Ez entlang mittlerer Linie",
    ylabel: str | None = None,
    save_path: str | None = None,
):
    """
    Plottet das elektrische Feld `Ez` entlang der horizontalen
    Mittellinie (y = Ny//2) für alle gespeicherten Zeitpunkte
    und speichert das Bild automatisch als PNG.

    Parameters
    ----------
    times : array‑like
        Zeitvektor (s) zu den jeweiligen Zeitschritten.
    Ez_line : 2‑D‑array
        Zeilen = verschiedene Zeitpunkte, Spalten = Gitter‑x‑Index.
    title, ylabel : optional
        Plot‑Beschriftungen.
    save_path : str | None
        Vollständiger Pfad inkl. Dateiname.  Wenn None → wird ein
        Name aus `title` im Unterordner `plots/` erzeugt.
    """
    if ylabel is None:
        ylabel = r"$E_z$ (a.u.)"

    plt.figure(figsize=(10, 5))

    # Maximal 6 Linien (gleichmäßig verteilt) für Übersichtlichkeit
    n_steps = len(times)
    step_skip = max(1, n_steps // 6)

    for idx, t in enumerate(times):
        if idx % step_skip == 0:
            plt.plot(
                np.arange(Ez_line.shape[1]),
                Ez_line[idx],
                label=f"{t * 1e9:.1f} ns",
            )

    plt.xlabel("Gitterindex (x)")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True, which="both", ls="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()

    # ----------- automatisches Speichern -------------
    out_file = _prepare_save_path(title, save_path)
    plt.savefig(out_file, dpi=300, bbox_inches="tight")
    print(f"[plot_field_line] Bild gespeichert → {out_file}")

    plt.show()


# ------------------------------------------------------------------
# 2‑D‑Snapshot – Speichert automatisch als PNG
# ------------------------------------------------------------------
def plot_field_snapshot(
    Ez,
    x_nodes,
    y_nodes,
    title: str = "Ez‑Snapshot (2‑D)",
    cmap: str = "RdBu_r",
    vmin: float | None = None,
    vmax: float | None = None,
    save_path: str | None = None,
):
    """
    2‑D‑Heat‑Map des elektrischen Feldes `Ez` und automatisches PNG‑Save.

    Parameters
    ----------
    Ez : 2‑D‑array (Nx × Ny)
        Feldwerte.
    x_nodes, y_nodes : 1‑D‑arrays
        Gitterkoordinaten (für die Extent‑Angabe).
    title, cmap, vmin, vmax : optional
        Plot‑Einstellungen.
    save_path : str | None
        Ziel‑Dateipfad.  Wenn None → Name aus `title` im Ordner `plots/`.
    """
    plt.figure(figsize=(8, 6))
    extent = [x_nodes[0], x_nodes[-1], y_nodes[0], y_nodes[-1]]
    im = plt.imshow(
        Ez.T,
        extent=extent,
        origin="lower",
        cmap=cmap,
        vmin=vmin,
        vmax=vmax,
        aspect="auto",
    )
    plt.colorbar(im, label=r"$E_z$ (a.u.)")
    plt.xlabel("x (m)")
    plt.ylabel("y (m)")
    plt.title(title)
    plt.tight_layout()

    # ----------- automatisches Speichern -------------
    out_file = _prepare_save_path(title, save_path)
    plt.savefig(out_file, dpi=300, bbox_inches="tight")
    print(f"[plot_field_snapshot] Bild gespeichert → {out_file}")

    plt.show()


# ------------------------------------------------------------------
# Energie‑Kurve – Speichert automatisch als PNG
# ------------------------------------------------------------------
def plot_energy(
    result_dict,
    title: str = "Gesamtenergie im Laufe der Simulation",
    save_path: str | None = None,
):
    """
    Zeigt die zeitliche Entwicklung der Gesamtenergie (E + H)
    und speichert das Bild automatisch als PNG.

    Parameters
    ----------
    result_dict : dict
        Rückgabe‑Dictionary von `run_simulation`.  Muss die Schlüssel
        ``time``, ``energy_hist``, ``dt`` und ``save_every`` enthalten.
    title : str, optional
        Plot‑Titel.
    save_path : str | None
        Ziel‑Dateipfad.  Wenn None → Name aus `title` im Ordner `plots/`.
    """
    energy_hist = result_dict.get("energy_hist")
    if energy_hist is None or len(energy_hist) == 0:
        print("⚠️  Keine Energiedaten vorhanden (DEBUG‑Modus aus?).")
        return

    # Zeitvektor (vollständig) und Schrittweite
    times = result_dict["time"]
    dt = result_dict["dt"]
    save_every = result_dict["save_every"]

    # Energie‑Zeit‑Achse: jedes gespeicherte Sample entspricht
    #   dt * save_every Sekunden.
    t_energy = np.arange(len(energy_hist)) * dt * save_every

    plt.figure(figsize=(10, 4))
    plt.plot(t_energy * 1e9, energy_hist, "-o")
    plt.xlabel("Time (ns)")
    plt.ylabel("Total energy (J)")
    plt.title(title)
    plt.grid(True, which="both", ls="--", alpha=0.5)
    plt.tight_layout()

    # ----------- automatisches Speichern -------------
    out_file = _prepare_save_path(title, save_path)
    plt.savefig(out_file, dpi=300, bbox_inches="tight")
    print(f"[plot_energy] Bild gespeichert → {out_file}")

    plt.show()


# ------------------------------------------------------------------
# Hüllkurven‑Analyse – Speichert optionales Spektrum‑Plot
# ------------------------------------------------------------------
def analyze_envelope(
    signal,
    name: str,
    dt,
    save_every,
    n_peaks: int = 5,
    do_plot: bool = True,
    save_path: str | None = None,
):
    """
    FFT‑Analyse einer 1‑D‑Zeitreihe (z. B. maxEz_list oder meanEz_list)
    und optionales Plot‑Speichern.

    Parameters
    ----------
    signal : 1‑D‑array
        Zeitreihe, die analysiert werden soll.
    name : str
        Label für Logging und (falls `do_plot=True`) für den Plot‑Titel.
    dt : float
        Grund‑Zeitschritt des FDTD‑Solvers.
    save_every : int
        Abstand der gespeicherten Samples (wie in `run_simulation`).
    n_peaks : int, optional
        Wie viele dominante Frequenz‑Peaks ausgegeben werden.
    do_plot : bool, optional
        Ob das Spektrum‑Plot erzeugt wird.
    save_path : str | None
        Ziel‑Dateipfad für das Spektrum‑Plot.  Wenn None → Name aus
        `name` im Ordner `plots/`.  Ignoriert, wenn `do_plot=False`.
    """
    signal = np.asarray(signal, dtype=float)

    # -------------------------------------------------
    # Effektive Abtastzeit / -frequenz der Hüllkurve
    # -------------------------------------------------
    dt_eff = dt * save_every
    fs_eff = 1.0 / dt_eff

    n = len(signal)
    t = np.arange(n) * dt_eff

    # DC‑Anteil entfernen + Hanning‑Fenster
    sig_centered = signal - np.mean(signal)
    window = np.hanning(n)
    sig_win = sig_centered * window

    # FFT (reell, nur positive Frequenzen)
    spec = np.fft.rfft(sig_win)
    freq = np.fft.rfftfreq(n, dt_eff)
    amp = np.abs(spec)

    # Null‑/DC‑Komponente ignorieren für Peak‑Suche
    amp_no_dc = amp.copy()
    amp_no_dc[0] = 0.0

    # stärkste Peaks finden
    idx_sorted = np.argsort(amp_no_dc)[::-1]
    idx_peaks = idx_sorted[:n_peaks]

    print(f"\n=== Hüllspektrum: {name} ===")
    print(f"Samples: {n}, dt_eff = {dt_eff:.3e} s, fs_eff = {fs_eff:.3e} Hz")
    print(f"Top {n_peaks} Spektralpeaks (nach Amplitude):")
    print("  Rank |   f_cps    |       f_Hz        |  Norm‑Amplitude")
    print("  -----+-----------+-------------------+----------------")
    amp_norm = amp / amp.max() if amp.max() > 0 else amp

    for rank, idx in enumerate(idx_peaks, start=1):
        f_hz = freq[idx]
        f_cps = f_hz / fs_eff
        print(f"  {rank:4d} | {f_cps:8.4f} | {f_hz:15.4e} | {amp_norm[idx]:.3f}")

    # -------------------------------------------------
    # Plot (falls gewünscht) + automatisches Speichern
    # -------------------------------------------------
    if do_plot:
        plt.figure(figsize=(10, 4))
        plt.plot(freq, amp, lw=1.0)
        plt.xlabel("Frequenz [Hz]")
        plt.ylabel("Amplitude")
        plt.title(f"Spektrum Hüllkurve: {name}")
        plt.grid(True)
        plt.tight_layout()

        # automatisches Speichern, wenn ein Pfad angegeben ist
        if save_path is not None:
            out_file = _prepare_save_path(name, save_path)
            plt.savefig(out_file, dpi=300, bbox_inches="tight")
            print(f"[analyze_envelope] Spektrum‑Plot gespeichert → {out_file}")

        plt.show()

    return freq, amp

In [7]:
# ------------------------------------------------------------
# Interaktive UI (mit Run‑Button) & Ausführung
# ------------------------------------------------------------

# ------------------------------------------------------------
# Sicherstellen, dass ipywidgets verfügbar ist
# ------------------------------------------------------------
try:
    HAVE_WIDGETS
except NameError:
    try:
        from ipywidgets import (
            IntSlider,
            FloatSlider,
            VBox,
            HBox,
            Output,
            interact_manual,
        )
        HAVE_WIDGETS = True
    except Exception:                      # pragma: no cover
        HAVE_WIDGETS = False

# ------------------------------------------------------------
# display‑Import (für die Ausgabe)
# ------------------------------------------------------------
from IPython.display import display

# ------------------------------------------------------------
# Hilfs‑Funktion für hübsche Debug‑Ausgabe
# ------------------------------------------------------------
def dbg_header(params: dict):
    """Formatiert die Parameter‑Übersicht für die Console."""
    print("\n=== Eingestellte Parameter (Slider‑Werte) ===")
    for key, val in params.items():
        if isinstance(val, float):
            # 4 Nachkommastellen für Floats (z. B. Courant, γ, α, A₀)
            print(f"{key:<22}= {val:.4f}")
        else:
            print(f"{key:<22}= {val}")
    print("=============================================\n")

# ------------------------------------------------------------
# Callback‑Funktion: Simulation + Plot‑Aufruf + Analyse
# ------------------------------------------------------------
def run_and_plot(
    Nx,
    Ny,
    courant,
    pml_cells,
    gamma_damping,
    feedback_interval,
    feedback_fraction,
    alpha,
    A0,
    save_every,
    arrival_factor,
):
    """
    Führt die Simulation mit den aktuellen Slider‑Werten aus,
    erzeugt die drei Standard‑Plots, führt die Hüllkurven‑Analyse
    durch und gibt die korrigierte Lichtgeschwindigkeit aus.
    """
    # ---- Parameter‑Log ----
    param_dict = {
        "Nx": Nx,
        "Ny": Ny,
        "Courant‑Faktor": courant,
        "PML‑Zellen": pml_cells,
        "Rand‑Dämpfung γ": gamma_damping,
        "Feedback‑Intervall": feedback_interval,
        "Feedback‑Fraktion": feedback_fraction,
        "α (Wachstums‑Rate)": alpha,
        "A₀ (V/m)": A0,
        "save_every": save_every,
        "arrival_factor": arrival_factor,
    }
    dbg_header(param_dict)

    # ---- Simulation starten ----
    # Alle Parameter (inkl. save_every & arrival_factor) werden jetzt
    # an run_simulation übergeben.
    result = run_simulation(
        Nx=Nx,
        Ny=Ny,
        courant=courant,
        pml_cells=pml_cells,
        gamma_damping=gamma_damping,
        alpha=alpha,
        A0=A0,
        feedback_interval=feedback_interval,
        feedback_fraction=feedback_fraction,
        save_every=save_every,          # <-- jetzt vom Slider
        arrival_factor=arrival_factor,  # <-- jetzt vom Slider
        debug=True,
    )

    # ---- Gitter‑Koordinaten für den Snapshot‑Plot ----
    # (wir benötigen die originalen x/y‑Knoten, nicht die interpolierten)
    x_nodes, _ = make_log_grid(1e-3, 30.0, Nx)
    y_nodes, _ = make_log_grid(1e-3, 30.0, Ny)

    # ---- Plots erzeugen (automatisches PNG‑Save) ----
    plot_field_line(
        result["time"],
        result["Ez_line"],
        title="Ez‑Linie_(Mittellinie)",
    )

    plot_field_snapshot(
        result["Ez_snapshot"],
        x_nodes,
        y_nodes,
        title="Ez‑Snapshot_(2‑D)",
    )

    plot_energy(
        result,
        title="Gesamtenergie_(E+H)",
    )

    # ---- Hüllkurven‑Analyse (automatisches PNG‑Save) ----
    analyze_envelope(
        result["maxEz_list"],
        name="maxEz",
        dt=result["dt"],
        save_every=result["save_every"],
        n_peaks=5,
        do_plot=True,
        save_path="plots/Hüllkurve_maxEz.png",
    )
    analyze_envelope(
        result["meanEz_list"],
        name="meanEz",
        dt=result["dt"],
        save_every=result["save_every"],
        n_peaks=5,
        do_plot=True,
        save_path="plots/Hüllkurve_meanEz.png",
    )

    # ---- Ausgabe korrigierter Geschwindigkeit (falls berechnet) ----
    import numpy as np
    if not np.isnan(result.get("c_corr", np.nan)):
        print("\n=== Geschwindigkeits‑Korrektur (nach Projektions‑Faktor) ===")
        print(f"c_corr (m/s)          = {result['c_corr']:.3e}")
        print(f"Relative Abweichung   = {result['rel_dev_corr']:.3f} %")
        print(f"Projektions‑Faktor Γ = {result['Gamma']:.3e}")
    else:
        print("\n⚠️  Keine Ankunftszeit ermittelt – Geschwindigkeits‑Korrektur nicht möglich.")

# ------------------------------------------------------------
# UI‑Layout (Slider‑Panel)
# ------------------------------------------------------------
if HAVE_WIDGETS:
    style = {"description_width": "120px"}

    # ------------------- Slider‑Definitionen -------------------
    Nx_slider = IntSlider(
        min=90,
        max=600,
        step=10,
        value=560,
        description="Nx (x‑Gitter)",
        style=style,
    )
    Ny_slider = IntSlider(
        min=90,
        max=500,
        step=10,
        value=180,
        description="Ny (y‑Gitter)",
        style=style,
    )
    courant_slider = FloatSlider(
        min=0.001,
        max=0.200,
        step=0.005,
        value=0.056,
        description="Courant‑Faktor",
        style=style,
        readout_format=".4f",
    )
    pml_slider = IntSlider(
        min=10,
        max=100,
        step=1,
        value=80,
        description="PML‑Zellen",
        style=style,
    )
    gamma_slider = FloatSlider(
        min=0.800,
        max=0.999,
        step=0.001,
        value=0.997,
        description="Rand‑Dämpfung γ",
        style=style,
        readout_format=".4f",
    )
    fb_int_slider = IntSlider(
        min=0,
        max=20000,
        step=200,
        value=0,
        description="Feedback‑Intervall",
        style=style,
    )
    fb_frac_slider = FloatSlider(
        min=0.0,
        max=0.5,
        step=0.01,
        value=0.0,
        description="Feedback‑Fraktion",
        style=style,
    )
    alpha_slider = FloatSlider(
        min=0.01,
        max=0.30,
        step=0.01,
        value=0.05,
        description="α (Wachstums‑Rate)",
        style=style,
    )
    A0_slider = FloatSlider(
        min=0.001,
        max=0.05,
        step=0.001,
        value=0.05,
        description="A₀ (V/m)",
        style=style,
        readout_format=".4f",
    )
    # ---- neue Slider für save_every und arrival_factor ----
    save_every_slider = IntSlider(
        min=20,
        max=200,
        step=10,
        value=120,
        description="save_every",
        style=style,
    )
    arrival_factor_slider = FloatSlider(
        min=0.01,
        max=0.20,
        step=0.01,
        value=0.05,
        description="Arrival‑Factor",
        style=style,
        readout_format=".3f",
    )

    # UI‑Panel zusammenbauen
    ui = VBox(
        [
            HBox([Nx_slider, Ny_slider]),
            HBox([courant_slider, pml_slider]),
            HBox([gamma_slider, fb_int_slider, fb_frac_slider]),
            HBox([alpha_slider, A0_slider]),
            HBox([save_every_slider, arrival_factor_slider]),
        ]
    )

    # Interaktive Bindung mit Run‑Button
    interact_manual(
        run_and_plot,
        Nx=Nx_slider,
        Ny=Ny_slider,
        courant=courant_slider,
        pml_cells=pml_slider,
        gamma_damping=gamma_slider,
        feedback_interval=fb_int_slider,
        feedback_fraction=fb_frac_slider,
        alpha=alpha_slider,
        A0=A0_slider,
        save_every=save_every_slider,
        arrival_factor=arrival_factor_slider,
    )
else:
    # ------------------------------------------------------------
    # Fallback: kein ipywidgets → einfaches Beispiel ausführen
    # ------------------------------------------------------------
    dbg_print("⚠️  ipywidgets nicht verfügbar – führe Beispiel‑Simulation aus.")

    example = run_simulation(
        Nx=150,
        Ny=150,
        courant=0.12,
        pml_cells=20,
        gamma_damping=0.90,
        feedback_interval=0,
        feedback_fraction=0.0,
        debug=True,
    )

    # Gitter‑Koordinaten für den Snapshot‑Plot
    x_nodes, _ = make_log_grid(1e-3, 30.0, 150)
    y_nodes, _ = make_log_grid(1e-3, 30.0, 150)

    # Plots – exakt dieselbe Reihenfolge wie im UI‑Workflow
    plot_field_line(example["time"], example["Ez_line"])
    plot_field_snapshot(example["Ez_snapshot"], x_nodes, y_nodes)
    plot_energy(example)

    # Hüllkurven‑Analyse für das Beispiel
    analyze_envelope(
        example["maxEz_list"],
        name="maxEz",
        dt=example["dt"],
        save_every=example["save_every"],
        n_peaks=5,
        do_plot=True,
        save_path="plots/maxEz.png",
    )
    analyze_envelope(
        example["meanEz_list"],
        name="meanEz",
        dt=example["dt"],
        save_every=example["save_every"],
        n_peaks=5,
        do_plot=True,
        save_path="plots/meanEz.png",
    )

interactive(children=(IntSlider(value=560, description='Nx (x‑Gitter)', max=600, min=90, step=10, style=Slider…